# Initiate spark

In [1]:
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = (
    SparkConf()
    .setAppName("Spark minIO Test")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
#sqlContext = SQLContext(sc)
spark = SparkSession(sc).builder.getOrCreate()

21/08/31 07:09:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [25]:
from delta.tables import DeltaTable

In [2]:
spark.sql(f"""
DROP TABLE IF EXISTS test_bronze_knmi_daggegevens
""")

spark.sql(f"""
CREATE TABLE test_bronze_knmi_daggegevens
USING DELTA
LOCATION 's3a://bronze-knmi/daggegevens'
""")

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.821/08/31 07:10:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[]

In [13]:
from_path = 's3a://bronze-knmi/daggegevens'
to_path = 's3a://bronze-knmi/daggegevens_bu'
numFiles = 16

spark.read \
 .format("delta") \
 .load(from_path) \
 .repartition(numFiles) \
 .write \
 .format("delta") \
 .mode("overwrite") \
 .save(to_path) 

In [7]:
from_path = 's3a://bronze-knmi/daggegevens'
to_path = 's3a://bronze-knmi/daggegevens_bu'
numFiles = 16

spark.read \
 .format("delta") \
 .load(to_path) \
 .repartition(numFiles) \
 .write \
 .format("delta") \
 .mode("overwrite") \
 .save(from_path) 

In [20]:
spark.sql(f"""
DROP TABLE IF EXISTS test_bronze_knmi_daggegevens
""")

spark.sql(f"""
CREATE TABLE test_bronze_knmi_daggegevens
USING DELTA
LOCATION 's3a://bronze-knmi/daggegevens'
""")

spark.sql("""
DELETE FROM test_bronze_knmi_daggegevens
""")

DataFrame[]

In [3]:
spark.sql("""
Select count(*) from test_bronze_knmi_daggegevens
""").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [ ]:
spark.sql("""
Vacuum test_bronze_knmi_daggegevens
""").show()